In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from tqdm import tqdm
import math

#Torch-related imports
import torch
import torch.distributions as D
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Function

#Model-specific imports
from SBM_SDE import *
from obs_and_flow import *
from training import calc_log_lik

In [2]:
torch.manual_seed(0)
np.random.seed(0)

In [3]:
temp_ref = 283
temp_rise = 5 #High estimate of 5 celsius temperature rise by 2100. 

#System parameters from deterministic CON model
u_M = 0.002
a_SD = 0.33
a_DS = 0.33
a_M = 0.33
a_MSC = 0.5
k_S_ref = 0.000025
k_D_ref = 0.005
k_M_ref = 0.0002
Ea_S = 75
Ea_D = 50
Ea_M = 50

#SCON diffusion matrix parameters
c_SOC = 0.1
c_DOC = 0.0001
c_MBC = 0.001
s_SOC = 0.001
s_DOC = 0.001
s_MBC = 0.001

SCON_C_params_dict = {'u_M': u_M, 'a_SD': a_SD, 'a_DS': a_DS, 'a_M': a_M, 'a_MSC': a_MSC, 'k_S_ref': k_S_ref, 'k_D_ref': k_D_ref, 'k_M_ref': k_M_ref, 'Ea_S': Ea_S, 'Ea_D': Ea_D, 'Ea_M': Ea_M, 'c_SOC': c_SOC, 'c_DOC': c_DOC, 'c_MBC': c_MBC}
SCON_SS_params_dict = {'u_M': u_M, 'a_SD': a_SD, 'a_DS': a_DS, 'a_M': a_M, 'a_MSC': a_MSC, 'k_S_ref': k_S_ref, 'k_D_ref': k_D_ref, 'k_M_ref': k_M_ref, 'Ea_S': Ea_S, 'Ea_D': Ea_D, 'Ea_M': Ea_M, 's_SOC': s_SOC, 's_DOC': s_DOC, 's_MBC': s_MBC}

#System parameters from deterministic AWB model
u_Q_ref = 0.2
Q = 0.002
a_MSA = 0.5
K_D = 200
K_U = 1
V_D_ref = 0.4
V_U_ref = 0.02
Ea_V_D = 75
Ea_V_U = 50
r_M = 0.0004
r_E = 0.00001
r_L = 0.0005

#SAWB diffusion matrix parameters
c_SOC = 2
c_DOC = 0.05
c_MBC = 0.1
c_EEC = 0.01
s_SOC = 0.1
s_DOC = 0.1
s_MBC = 0.1
s_EEC = 0.1

SAWB_C_params_dict = {'u_Q_ref': u_Q_ref, 'Q': Q, 'a_MSA': a_MSA, 'K_D': K_D, 'K_U': K_U, 'V_D_ref': V_D_ref, 'V_U_ref': V_U_ref, 'Ea_V_D': Ea_V_D, 'Ea_V_U': Ea_V_U, 'r_M': r_M, 'r_E': r_E, 'r_L': r_L, 'c_SOC': c_SOC, 'c_DOC': c_DOC, 'c_MBC': c_MBC, 'c_EEC': c_EEC}
SAWB_SS_params_dict = {'u_Q_ref': u_Q_ref, 'Q': Q, 'a_MSA': a_MSA, 'K_D': K_D, 'K_U': K_U, 'V_D_ref': V_D_ref, 'V_U_ref': V_U_ref, 'Ea_V_D': Ea_V_D, 'Ea_V_U': Ea_V_U, 'r_M': r_M, 'r_E': r_E, 'r_L': r_L, 's_SOC': s_SOC, 's_DOC': s_DOC, 's_MBC': s_MBC, 's_EEC': s_EEC}

#System parameters from deterministic AWB-ECA model
u_Q_ref = 0.2
Q = 0.002
a_MSA = 0.5
K_DE = 200
K_UE = 1
V_DE_ref = 0.4
V_UE_ref = 0.02
Ea_V_DE = 75
Ea_V_UE = 50
r_M = 0.0004
r_E = 0.00001
r_L = 0.0005

#SAWB-ECA diffusion matrix parameters
c_SOC = 2
c_DOC = 0.05
c_MBC = 0.1
c_EEC = 0.01
s_SOC = 0.1
s_DOC = 0.1
s_MBC = 0.1
s_EEC = 0.1

SAWB_ECA_C_params_dict = {'u_Q_ref': u_Q_ref, 'Q': Q, 'a_MSA': a_MSA, 'K_DE': K_DE, 'K_UE': K_UE, 'V_DE_ref': V_DE_ref, 'V_UE_ref': V_UE_ref, 'Ea_V_DE': Ea_V_DE, 'Ea_V_UE': Ea_V_UE, 'r_M': r_M, 'r_E': r_E, 'r_L': r_L, 'c_SOC': c_SOC, 'c_DOC': c_DOC, 'c_MBC': c_MBC, 'c_EEC': c_EEC}
SAWB_ECA_SS_params_dict = {'u_Q_ref': u_Q_ref, 'Q': Q, 'a_MSA': a_MSA, 'K_DE': K_DE, 'K_UE': K_UE, 'V_DE_ref': V_DE_ref, 'V_UE_ref': V_UE_ref, 'Ea_V_DE': Ea_V_DE, 'Ea_V_UE': Ea_V_UE, 'r_M': r_M, 'r_E': r_E, 'r_L': r_L, 's_SOC': s_SOC, 's_DOC': s_DOC, 's_MBC': s_MBC, 's_EEC': s_EEC}

In [4]:
#Set flow NN parameters.

devi = torch.device("".join(["cuda:",f'{cuda_id}']) if torch.cuda.is_available() else "cpu")
dt_flow = 0.2
t = 1000
n_flow = int(t / dt_flow) + 1
t_span = np.linspace(0, t, n_flow)
t_span_tensor = torch.reshape(torch.Tensor(t_span), [1, n_flow, 1]) #T_span needs to be converted to tensor object. Additionally, facilitates conversion of I_S and I_D to tensor objects.
l_r = 1e-4
niter = 5001
piter = 101
batch_size = 3 #Number of sets of observation outputs to sample per set of parameters.
state_dim_SCON = 3 #Not including CO2 in STATE_DIM, because CO2 is an observation.
obs_error_scale = 0.1 #Proportion of the mean of observation error standard deviation.

x0_SCON = [19, 0.08, 0.8] #Initial condition means for SCON

In [5]:
#Obtain temperature forcing function.
temp_tensor = temp_gen(t_span_tensor, temp_ref, temp_rise)
print(temp_tensor)

#Obtain SOC and DOC pool litter input vectors for use in flow SDE functions.
i_s_tensor = i_s(t_span_tensor) #Exogenous SOC input function
i_d_tensor = i_d(t_span_tensor) #Exogenous DOC input function
print(i_s_tensor)
print(i_d_tensor)

tensor([[[283.0000],
         [283.5248],
         [284.0482],
         ...,
         [281.4880],
         [281.1925],
         [280.9200]]])
tensor([[[0.0010],
         [0.0010],
         [0.0010],
         ...,
         [0.0013],
         [0.0013],
         [0.0013]]])
tensor([[[1.0000e-04],
         [1.0001e-04],
         [1.0001e-04],
         ...,
         [1.3286e-04],
         [1.3286e-04],
         [1.3287e-04]]])


In [6]:
def train(DEVICE, L_R, NITER, PRETRAIN_ITER, BATCH_SIZE, SDEFLOW, ObsModel, csv_to_obs_df, DATA_CSV, OBS_ERROR_SCALE, STATE_DIM, T, DT, N, T_SPAN_TENSOR, I_S_TENSOR, I_D_TENSOR, TEMP_TENSOR, TEMP_REF, C0, DRIFT_DIFFUSION, PARAMS_DICT): 
    #Read-in observation information. 
    obs_times, obs_means, obs_error = csv_to_obs_df(DATA_CSV, STATE_DIM, T, OBS_ERROR_SCALE)
    obs_means = LowerBound.apply(obs_means, 1e-6)
    #Pass observation information to `ObsModel`.
    obs_model = ObsModel(DEVICE, obs_times, DT, obs_means, obs_error)
    net = SDEFlow(DEVICE, obs_model, STATE_DIM, T, DT, N, I_S_TENSOR, I_D_TENSOR, cond_inputs = 3, num_layers = 6).to(DEVICE)
    optimizer = optim.Adam(net.parameters(), lr = L_R)
    if PRETRAIN_ITER >= NITER:
        raise Exception("PRETRAIN_ITER must be < NITER.")
    best_loss_norm = 1e15
    best_loss_ELBO = 1e15
    norm_losses = []
    ELBO_losses = []
    C0_tensor = torch.tensor(C0).to(DEVICE) #Convert initial conditions from list to tensor for X0 prior object.
    #C0 = C0[(None,) * 2].repeat(BATCH_SIZE, 1, 1).to(DEVICE)
    PARAMS_DICT_TENSOR = {k: torch.tensor(v).expand(BATCH_SIZE) for k, v in PARAMS_DICT.items()}
    X0_prior = D.normal.Normal(loc = C0_tensor, scale = OBS_ERROR_SCALE * C0_tensor) #Setting prior noise = observation noise for now.
    with tqdm(total = NITER, desc = f'Train Diffusion', position = -1) as tq:
        for i in range(NITER):
            net.train()
            optimizer.zero_grad()
            C_PATH, log_prob = net(BATCH_SIZE) #For obs_and_flow.py
            #C_PATH = torch.cat([C0, C_PATH], 1) #Learning initial conditions in this version. #Append deterministic CON initial conditions conditional on parameter values to C path.
            if i <= PRETRAIN_ITER:
                l1_norm_element = C_PATH - torch.mean(obs_model.mu, -1)
                l1_norm = torch.sum(torch.abs(l1_norm_element)).mean()
                best_loss_norm = l1_norm if l1_norm < best_loss_norm else best_loss_norm
                norm_losses.append(l1_norm.item())
                #l2_norm_element = C_PATH - torch.mean(obs_model.mu, -1)
                #l2_norm = torch.sqrt(torch.sum(torch.square(l2_norm_element))).mean()
                #best_loss_norm = l2_norm if l2_norm < best_loss_norm else best_loss_norm
                #norm_losses.append(l2_norm.item())
                if i % 10 == 0:
                    ma_norm_loss = sum(norm_losses[-10:]) / len(norm_losses[-10:])
                    print(f"\nMoving average norm loss at {iter} iterations is: {ma_norm_loss}. Best norm loss value is: {best_loss_norm}.")
                    print('\nC_PATH mean =', C_PATH.mean(-2))
                    print('\nC_PATH =', C_PATH)
                l1_norm.backward()
                #l2_norm.backward()
            else:
                log_lik = calc_log_lik(C_PATH, T_SPAN_TENSOR.to(DEVICE), DT, I_S_TENSOR.to(DEVICE), I_D_TENSOR.to(DEVICE), TEMP_TENSOR.to(DEVICE), TEMP_REF, DRIFT_DIFFUSION, PARAMS_DICT)
                neg_ELBO = -X0_prior.log_prob(C_PATH[:, 0, :]).sum(-1).mean() - log_lik.mean() - obs_model(C_PATH, PARAMS_DICT_TENSOR) + log_prob.mean()
                best_loss_ELBO = neg_ELBO if neg_ELBO < best_loss_ELBO else best_loss_ELBO
                ELBO_losses.append(neg_ELBO.item())
                if i % 10 == 0:             
                    ma_elbo_loss = sum(ELBO_losses[-10:]) / len(ELBO_losses[-10:])
                    print(f"\nMoving average ELBO loss at {iter} iterations is: {ma_elbo_loss}. Best ELBO loss value is: {best_loss_ELBO}.")
                    print('\nC_PATH mean =', C_PATH.mean(-2))
                    print('\nC_PATH =', C_PATH)
            torch.nn.utils.clip_grad_norm_(net.parameters(), 3.0)
            optimizer.step()
            if i % 100000 == 0 and i > 0:
                optimizer.param_groups[0]['lr'] *= 0.1
            tq.update()
    return net, ELBO_losses, norm_losses

In [7]:
net, ELBO_losses, norm_losses = train(devi, l_r, niter, piter, batch_size, SDEFlow, ObsModel, csv_to_obs_df, 'y_from_x_t_1000_dt_0-01.csv', obs_error_scale, state_dim_SCON, t, dt_flow, n_flow, t_span_tensor, i_s_tensor, i_d_tensor, temp_tensor, temp_ref, x0_SCON, drift_diffusion_SCON_C, SCON_C_params_dict)


Train Diffusion:   0%|          | 0/5001 [00:00<?, ?it/s]


Moving average norm loss at <built-in function iter> iterations is: 487776.46875. Best norm loss value is: 487776.46875.

C_PATH mean = tensor([[0.8412, 0.8549, 0.8230],
        [0.8406, 0.8506, 0.8430],
        [0.8338, 0.8452, 0.8374]], grad_fn=<MeanBackward1>)

C_PATH = tensor([[[0.7979, 0.4989, 0.8650],
         [1.0883, 0.3769, 2.7311],
         [0.3203, 0.7267, 1.9741],
         ...,
         [0.5538, 0.7271, 0.4697],
         [0.6251, 1.0342, 0.5232],
         [0.5830, 1.1936, 0.6196]],

        [[0.7932, 0.9848, 0.8966],
         [0.2745, 1.7503, 1.1032],
         [0.2097, 0.5167, 1.5775],
         ...,
         [0.7713, 1.1644, 0.7867],
         [0.6278, 0.5162, 0.7906],
         [0.8528, 0.6983, 0.7032]],

        [[0.5078, 1.8464, 0.8937],
         [1.3542, 0.9721, 0.2880],
         [3.1418, 1.7542, 0.5738],
         ...,
         [1.0607, 0.4409, 0.7835],
         [1.4567, 0.4050, 0.8300],
         [1.0807, 0.6015, 0.7105]]], grad_fn=<AddBackward0>)



Train Diffusion:   0%|          | 10/5001 [01:29<11:54:39,  8.59s/it]


Moving average norm loss at <built-in function iter> iterations is: 486311.496875. Best norm loss value is: 485398.75.

C_PATH mean = tensor([[0.9002, 0.8340, 0.8882],
        [0.8929, 0.8188, 0.8916],
        [0.8892, 0.8296, 0.8777]], grad_fn=<MeanBackward1>)

C_PATH = tensor([[[0.9859, 1.3254, 1.4777],
         [0.4882, 1.3655, 1.1026],
         [1.6296, 0.5975, 1.0450],
         ...,
         [0.6678, 1.7428, 1.4339],
         [0.3964, 1.8624, 1.2646],
         [2.9912, 1.0947, 1.4510]],

        [[0.6973, 0.7329, 0.6936],
         [0.7490, 0.6953, 1.5204],
         [1.1318, 0.8571, 1.4014],
         ...,
         [0.5059, 0.8771, 0.6433],
         [0.9966, 0.6296, 0.6261],
         [0.7703, 0.8904, 1.1772]],

        [[0.6312, 0.7424, 0.7368],
         [1.1397, 0.7034, 0.4623],
         [0.2655, 1.1055, 0.7856],
         ...,
         [2.2622, 0.2000, 0.6261],
         [2.1634, 0.3482, 0.7702],
         [0.5345, 0.7446, 0.2625]]], grad_fn=<AddBackward0>)



Train Diffusion:   0%|          | 20/5001 [02:53<12:16:49,  8.88s/it]

...


Moving average ELBO loss at <built-in function iter> iterations is: 329716166.4. Best ELBO loss value is: 210565504.0.

C_PATH mean = tensor([[8.6465, 0.5071, 1.2959],
        [9.0460, 0.4912, 1.3114],
        [8.6997, 0.5070, 1.3022]], grad_fn=<MeanBackward1>)

C_PATH = tensor([[[6.9666e-01, 9.5825e-01, 8.5641e-01],
         [6.3221e-01, 7.3692e-01, 1.8727e+00],
         [2.7227e+01, 2.1945e-01, 1.2028e+00],
         ...,
         [3.2503e+01, 1.6473e-01, 1.0937e+00],
         [4.4649e+00, 2.4227e-01, 8.3479e-01],
         [8.2812e-01, 1.5161e+00, 1.6230e+00]],

        [[1.6030e+00, 8.1272e-01, 4.4611e+00],
         [1.1155e+00, 6.2089e-01, 1.0043e+00],
         [6.7803e-01, 7.6031e-01, 1.1510e+00],
         ...,
         [5.5966e-01, 7.0892e-01, 9.8665e-01],
         [3.2254e-01, 1.5099e+01, 5.0082e+00],
         [3.1594e-01, 4.3524e+00, 1.9831e+00]],

        [[6.0371e-01, 9.5396e-01, 8.4569e-01],
         [1.1904e+01, 7.1007e-01, 9.4913e-01],
         [1.8879e+00, 4.2518e-01, 7.6


Train Diffusion: 100%|█████████▉| 4991/5001 [3:44:02<00:26,  2.61s/it]


Moving average ELBO loss at <built-in function iter> iterations is: 324193414.4. Best ELBO loss value is: 210565504.0.

C_PATH mean = tensor([[8.7647, 0.4818, 1.2621],
        [8.6985, 0.5128, 1.2785],
        [8.7969, 0.4749, 1.3027]], grad_fn=<MeanBackward1>)

C_PATH = tensor([[[ 1.1987,  0.9052,  0.8421],
         [ 1.3448,  0.4271,  1.4641],
         [19.6975,  0.3536,  1.2068],
         ...,
         [ 6.7130,  0.4560,  1.9802],
         [ 1.1190,  0.5215,  0.8123],
         [25.0939,  1.2531,  2.0288]],

        [[ 1.2689,  0.8917,  3.3664],
         [ 1.0271,  1.0307,  1.3662],
         [ 0.7086,  1.0512,  0.5864],
         ...,
         [ 1.2154,  0.2137,  0.5004],
         [ 3.8915,  0.1100,  0.7722],
         [ 0.6003,  1.8829,  0.8345]],

        [[ 0.4437,  0.9539,  1.0193],
         [11.0355,  0.5043,  1.5724],
         [ 1.8704,  0.4087,  0.6368],
         ...,
         [ 0.2473,  0.5465,  1.0783],
         [41.5746,  0.0933,  1.3086],
         [ 5.9820,  0.3418,  1.0661


Train Diffusion: 100%|██████████| 5001/5001 [3:44:28<00:00,  2.69s/it]


Moving average ELBO loss at <built-in function iter> iterations is: 320832684.8. Best ELBO loss value is: 210565504.0.

C_PATH mean = tensor([[8.6869, 0.4912, 1.2754],
        [8.7156, 0.4874, 1.2909],
        [8.9660, 0.4817, 1.3114]], grad_fn=<MeanBackward1>)

C_PATH = tensor([[[ 0.5203,  0.9535,  0.9292],
         [ 0.3857,  0.9490,  1.0134],
         [20.7611,  0.3062,  1.0621],
         ...,
         [32.6309,  0.1232,  1.1992],
         [ 5.0736,  0.1318,  0.8055],
         [ 0.6420,  1.7724,  1.7481]],

        [[ 1.5472,  0.8197,  1.1879],
         [ 1.7845,  0.3387,  0.7105],
         [ 0.7498,  0.8070,  1.2796],
         ...,
         [ 0.3572,  0.6154,  1.0028],
         [38.5484,  0.0912,  1.3430],
         [ 5.7985,  0.3770,  1.2967]],

        [[ 0.8376,  0.9635,  1.2318],
         [12.0549,  0.3310,  1.2820],
         [ 2.9431,  0.3657,  0.8839],
         ...,
         [ 3.0455,  0.2460,  0.8575],
         [ 0.5711,  0.5401,  0.7778],
         [24.4659,  1.2421,  1.9994